In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class StockMarketEnv(gym.Env):
    def __init__(self, max_inventory=20):
        self.action_space = [0, 1, 2] # 0: 구매, 1: 판매, 2: 유지
        self.max_inventory=max_inventory # 최대 보유개수
        # self.observation_space = []
        
    def reset(self):
        self.current_price=1000 # 현재가
        self.purchase=10000 # 소지금
        self.before_purchase=1000
        self.purchased_prices=[0 for i in range(self.max_inventory)] # 구매한 코인들
        self.state=[self.current_price, self.purchase, self.purchased_prices] # [현재가, 소지금, 보유코인]
        return self.state, {}
    
    def step(self, action):
        self.current_price+=np.random.choice([-1,0,1])

        if action == 0: # 구매
            if self.purchase-self.current_price>=0: # 소지금이 현재가보다 많으면
                self.purchased_prices-=self.current_price # 소지금 -= 현재가 
                self.purchased_prices.append(self.current_price) # 보유코인에 현재가격 추가, 나중에 여기서 제일 비싼거로 팖
            